# Support vector regression 

In [1]:
import sys
ver_info = sys.version
print("This jupyter notebook was executed with:\n{0}".format(ver_info));


This jupyter notebook was executed with:
2.7.14 |Anaconda, Inc.| (default, Nov 20 2017, 18:04:19) 
[GCC 7.2.0]


## Imports

In [2]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats, integrate
import seaborn as sns

import pprint
p = pprint.PrettyPrinter(indent=4)

import config as config

%matplotlib inline


## utilities and dataset
+ Imports untilities and generates the dataset as pandas Dataframes 
+ Check config.py for the training/testing set ratio (currently TRAIN_FRAC = 0.75) 

In [3]:
%run utilities.py
util = Utilities({}, use_json=False)
training_list, testing_list = util.get_datasets()

# turning the datasets into pandas DataFrames 
training_DF, testing_DF = util.create_dataframes(training_list, testing_list)

# generate the energy and dance measures 
training_DF, testing_DF = util.generate_energy_measure(training_DF, testing_DF)
training_DF, testing_DF = util.generate_dance_measure(training_DF, testing_DF)


/home/rshn/anaconda3/envs/python2/lib/python2.7/site-packages/pandas/core/computation/check.py:17: UserWarning: The installed version of numexpr 2.2.2 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


## Making svr learners

In [4]:
from sklearn.svm import SVR
from sklearn import metrics 
# Look at utilities.py for RunAndTestLinearRegModel() 

In [5]:
%run utilities.py
def printerWrapper(input_string_list, train=False, test=False):
    
    for item in input_string_list:
        X_cols = item
        results = util.RunAndTestSVRModel(X_cols, training_DF, testing_DF)
        #  [x][1] = Mean squared error regression loss (bc its the smallest value lol)
        mserr_training = results[0]
        mserr_testing = results[1]
        if train: 
#             print "{:<40}".format(item), "training_error_rbf:\t", mserr_training[0]
            print "{:<40}".format(item), "training_error_lin:\t", mserr_training[1]
#             print "{:<40}".format(item), "training_error_poly:\t", mserr_training[2]
        if test:
#             print "{:<40}".format(item), "testing_error_rbf:\t\t\t", mserr_testing[0]
            print "{:<40}".format(item), "testing_error_lin:\t\t\t", mserr_testing[1]
#             print "{:<40}".format(item), "testing_error_poly:\t\t\t", mserr_testing[2]
        # print "The min error values were:", min(results[1]), min(results[2])
        # print "std of hot is:", results[3]
        # -------------------------------------------------


### Using metadata (art_hot and art_fam)

In [14]:
metadata = ['artist_familiarity', 'artist_hotttnesss', 'artist_hotttnesss artist_familiarity']
printerWrapper(metadata, False, True)

artist_familiarity                       testing_error_lin:			0.0207823330334
artist_hotttnesss                        testing_error_lin:			0.0221534696807
artist_hotttnesss artist_familiarity     testing_error_lin:			0.0206913126232


### using acoustic features

In [7]:
allsegs = 'segments_avg tatums_avg beats_avg bars_avg sections_avg '
allrawacous = 'key loudness duration tempo time_signature'

raw_acous = []
for each in allsegs.split():
    raw_acous.append(each)

raw_acous.append(allsegs)


for each in allrawacous.split():
    raw_acous.append(each)   
raw_acous.append(allrawacous)
raw_acous.append((allrawacous+ ' ' + allsegs))

printerWrapper(raw_acous, True, True)


segments_avg                             training_error_lin:	0.028380827084
segments_avg                             testing_error_lin:			0.0281333776869
tatums_avg                               training_error_lin:	0.0281005834203
tatums_avg                               testing_error_lin:			0.027818376399
beats_avg                                training_error_lin:	0.0281014890502
beats_avg                                testing_error_lin:			0.0281330018524
bars_avg                                 training_error_lin:	0.0283467300269
bars_avg                                 testing_error_lin:			0.0281307474524
sections_avg                             training_error_lin:	0.02833380791
sections_avg                             testing_error_lin:			0.0281769396472
segments_avg tatums_avg beats_avg bars_avg sections_avg  training_error_lin:	0.027862563086
segments_avg tatums_avg beats_avg bars_avg sections_avg  testing_error_lin:			0.0277782635166
key                                      tr

### Find best energy measure

In [8]:
energylsit = ['energy1', 'energy2', 'energy3', 'energy4', 'energy1 energy2 energy3 energy4']
printerWrapper(energylsit, True, True)



energy1                                  training_error_lin:	0.0213848348243
energy1                                  testing_error_lin:			0.0208953239173
energy2                                  training_error_lin:	0.0221882118538
energy2                                  testing_error_lin:			0.0222673200505
energy3                                  training_error_lin:	0.0217659300591
energy3                                  testing_error_lin:			0.020998582028
energy4                                  training_error_lin:	0.0235614575931
energy4                                  testing_error_lin:			0.0231838824006
energy1 energy2 energy3 energy4          training_error_lin:	0.0213338199646
energy1 energy2 energy3 energy4          testing_error_lin:			0.0209878456166


### dance now

In [9]:
dancelist = ['dance1', 'dance2', 'dance3', 'dance4', 'dance1 dance2 dance3 dance4']

printerWrapper(dancelist, True, True)

dance1                                   training_error_lin:	0.0229828024711
dance1                                   testing_error_lin:			0.0237196332055
dance2                                   training_error_lin:	0.0221998733703
dance2                                   testing_error_lin:			0.0230825418749
dance3                                   training_error_lin:	0.0241291890035
dance3                                   testing_error_lin:			0.0244062939743
dance4                                   training_error_lin:	0.0241820826992
dance4                                   testing_error_lin:			0.024481880518
dance1 dance2 dance3 dance4              training_error_lin:	0.0218754722092
dance1 dance2 dance3 dance4              testing_error_lin:			0.0229647884515


### All Acoustic

In [10]:
dancelist = 'dance1 dance2 dance3 dance4 '
energylist = 'energy1 energy2 energy3 energy4 '

acousitc =[]
acousitc.append(dancelist)
acousitc.append(energylist)
acousitc.append(dancelist+energylist)
acousitc.append("artist_familiarity artist_hotttnesss")
acousitc.append(dancelist+energylist+"artist_familiarity artist_hotttnesss")

printerWrapper(acousitc, True, True)


dance1 dance2 dance3 dance4              training_error_lin:	0.0218754722092
dance1 dance2 dance3 dance4              testing_error_lin:			0.0229647884515
energy1 energy2 energy3 energy4          training_error_lin:	0.0213338199646
energy1 energy2 energy3 energy4          testing_error_lin:			0.0209878456166
dance1 dance2 dance3 dance4 energy1 energy2 energy3 energy4  training_error_lin:	0.0204930947419
dance1 dance2 dance3 dance4 energy1 energy2 energy3 energy4  testing_error_lin:			0.0207891643069
artist_familiarity artist_hotttnesss     training_error_lin:	0.0205565154158
artist_familiarity artist_hotttnesss     testing_error_lin:			0.0206913126232
dance1 dance2 dance3 dance4 energy1 energy2 energy3 energy4 artist_familiarity artist_hotttnesss training_error_lin:	0.0201394821972
dance1 dance2 dance3 dance4 energy1 energy2 energy3 energy4 artist_familiarity artist_hotttnesss testing_error_lin:			0.0204195550606


### Final 

In [11]:
finalset = [dancelist+energylist+"artist_familiarity artist_hotttnesss"]

printerWrapper(finalset, True, True)


dance1 dance2 dance3 dance4 energy1 energy2 energy3 energy4 artist_familiarity artist_hotttnesss training_error_lin:	0.0201394821972
dance1 dance2 dance3 dance4 energy1 energy2 energy3 energy4 artist_familiarity artist_hotttnesss testing_error_lin:			0.0204195550606


In [12]:
rawacoustic = " key loudness duration tempo time_signature segments_avg tatums_avg beats_avg bars_avg sections_avg  "
above = ' dance1 dance2 dance3 dance4 energy1 energy2 energy3 energy4 artist_familiarity artist_hotttnesss  '
actual_all = rawacoustic+above

printerWrapper([actual_all], True, True)

 key loudness duration tempo time_signature segments_avg tatums_avg beats_avg bars_avg sections_avg   dance1 dance2 dance3 dance4 energy1 energy2 energy3 energy4 artist_familiarity artist_hotttnesss   training_error_lin:	0.0200916788824
 key loudness duration tempo time_signature segments_avg tatums_avg beats_avg bars_avg sections_avg   dance1 dance2 dance3 dance4 energy1 energy2 energy3 energy4 artist_familiarity artist_hotttnesss   testing_error_lin:			0.0205697598433
